In [28]:
import os
import cv2
import json
import pygame
import numpy as np
import csv

### Variables

In [29]:
# Ruta de la carpeta de imágenes
PATH = "Dataset/"

IMG_WIDTH = 2400 // 2
IMG_HEIGHT = 1350 // 2

BLACK = (0, 0, 0)

In [30]:
# Variables para controlar la visualización
current_image_index = 0
delete_bbox = False
deleted_bbox = []
new_bbox_start = None
new_bbox_end = None

### Cargar JSON

In [31]:
# Cargar el archivo JSON
with open('output_actualizado.json') as f:
    data = json.load(f)

In [32]:
# Obtener la lista de imágenes del JSON
images = data['images']
num_images = len(images)

### Cargar CSV (labels)

In [33]:
# Cargar los datos del archivo CSV en un diccionario
labels = {}
with open('dataset_caltech.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        file_path = row['path']
        label = row['class']
        labels[file_path] = label

### Funciones auxiliares

In [37]:
def draw_bounding_boxes(image, detections):
    for detection in detections:
        bbox = detection['bbox']
        x = int(bbox[0] * image.shape[1])
        y = int(bbox[1] * image.shape[0])
        w = int(bbox[2] * image.shape[1])
        h = int(bbox[3] * image.shape[0])
        category = detection['category']
        conf = detection['conf']

        if category != '1':
            color = (0, 255, 0)  # Verde para otras categorías
            conf_color = (0, 0, 0)  # Negro para la confianza
        else:
            color = (255, 0, 0)  # Rojo para la categoría '1'
            conf_color = (255, 255, 255)  # Blanco para la confianza

        # Dibujar la bounding box
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

        # Dibujar el rectángulo de la confianza
        conf_text = f'{conf:.2f}'
        (text_width, text_height), _ = cv2.getTextSize(conf_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(image, (x, y - text_height - 5), (x + text_width + 10, y - 5), color, cv2.FILLED)

        # Escribir la confianza encima del rectángulo
        cv2.putText(image, conf_text, (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, conf_color, 2)

    return image


### Cargar Progreso

In [38]:
def cargar_progreso():
    # Leer el número de imagen desde el archivo de texto
    if os.path.exists('progreso.txt'):
        with open('progreso.txt', 'r') as file:
            current_image_number = int(file.read())
    else:
        current_image_number = 0

    return current_image_number

In [39]:
# Inicializar Pygame
pygame.init()

# Variables para escribir texto
font = pygame.font.SysFont(None, 48)

current_image_number = cargar_progreso()

# Establecer el número de imagen actual
current_image_index = current_image_number

# Crear la ventana
screen = pygame.display.set_mode((IMG_WIDTH, IMG_HEIGHT), pygame.RESIZABLE)
pygame.display.set_caption('Visualizador de imágenes')

# Definir la variable img_label
img_label = 'unknown'

# Bucle principal
running = True
while running:
    # Obtener la imagen actual
    current_image = images[current_image_index]
    file = current_image['file']
    detections = current_image['detections']

    # Construir la ruta completa de la imagen
    image_path = os.path.join(PATH, file)

    # Obtener la etiqueta de clase correspondiente al nombre de archivo actual
    img_label = labels.get(file, 'unknown')

    # Verificar si la imagen tiene la etiqueta "empty"
    if img_label.lower() == "empty":
        # Si la etiqueta es "empty", eliminar todas las bounding boxes
        detections = []

    # Obtener el número actual de bounding boxes
    num_bboxes = len(detections)

    # Cargar la imagen con OpenCV
    #image = pygame.image.load(image_path)
    image = cv2.imread(image_path)

    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))

    # Dibujar las bounding boxes en la imagen
    image_with_boxes = draw_bounding_boxes(image, detections)

    # Mostrar la imagen en la ventana de Pygame
    image_rgb = cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB)

    #image_rotated = np.rot90(image_rgb, k=1)

    # Número de imagen
    n_img = font.render(str(current_image_index) + " / " + str(num_images), True, BLACK)

    img_label = font.render(str(img_label) + " (" + str(num_bboxes) + ")", True, BLACK)

    # String úmero de bounding boxes
    # bbox_count_text = font.render("Bounding Boxes: " + str(num_bboxes), True, BLACK)

    image_surface = pygame.surfarray.make_surface(np.rot90(np.fliplr(image_rgb)))
    #image_surface = pygame.surfarray.make_surface(image_rgb)
    screen.blit(image_surface, (0, 0))
    screen.blit(n_img, (IMG_WIDTH - IMG_WIDTH // 6, 20))
    screen.blit(img_label, (20, 20))
    # Mostrar el número de bounding boxes en la esquina inferior derecha
    # screen.blit(bbox_count_text, (IMG_WIDTH - bbox_count_text.get_width() - 10, IMG_HEIGHT - bbox_count_text.get_height() - 10))
    pygame.display.flip()

    # Variables para controlar la restauración del último elemento eliminado
    restored_bbox = None

    # Esperar eventos
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                current_image_index = (current_image_index - 1) % num_images
            elif event.key == pygame.K_RIGHT:
                current_image_index = (current_image_index + 1) % num_images
            elif event.key == pygame.K_DOWN:
                with open('toreview.txt', 'r') as file:
                    progreso = str(file.read())
                with open('toreview.txt', 'w') as file:
                    file.write(progreso + "," + str(current_image_index))
            elif event.key == pygame.K_x:
                delete_bbox = True
            elif event.key == pygame.K_z:
                if deleted_bbox:
                    restored_bbox = deleted_bbox.pop()
            elif event.key == pygame.K_s:
                with open('output_actualizado.json', 'w') as f:
                    json.dump(data, f, indent=2)
                with open('progreso.txt', 'w') as file:
                    file.write(str(current_image_index))
            '''elif event.key == pygame.K_PLUS or event.key == pygame.K_KP_PLUS:
                contrast += 0.1
            elif event.key == pygame.K_MINUS or event.key == pygame.K_KP_MINUS:
                contrast -= 0.1'''
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == pygame.BUTTON_RIGHT:
                if new_bbox_start is None:
                    # Primer click derecho, establecer el punto de inicio del cuadrado
                    new_bbox_start = event.pos
                else:
                    # Segundo click derecho, establecer el punto final del cuadrado y agregarlo a las bounding boxes
                    new_bbox_end = event.pos
                    x1, y1 = new_bbox_start
                    x2, y2 = new_bbox_end

                    # Normalizar las coordenadas del cuadrado
                    x_norm1 = x1 / screen.get_width()
                    y_norm1 = y1 / screen.get_height()
                    x_norm2 = x2 / screen.get_width()
                    y_norm2 = y2 / screen.get_height()

                    # Calcular las coordenadas de la bounding box en relación a la imagen
                    x = min(x_norm1, x_norm2)
                    y = min(y_norm1, y_norm2)
                    w = abs(x_norm2 - x_norm1)
                    h = abs(y_norm2 - y_norm1)

                    # Añadir la nueva bounding box al JSON de detecciones
                    detections.append({
                        'category': '1',
                        'conf': 0.0,
                        'bbox': [x, y, w, h]
                    })

                    # Reiniciar los puntos de inicio y final del cuadrado
                    new_bbox_start = None
                    new_bbox_end = None
            elif event.button == pygame.BUTTON_LEFT and delete_bbox:
                # Obtener las coordenadas del ratón
                x, y = event.pos

                # Normalizar las coordenadas del ratón
                x_norm = x / screen.get_width()
                y_norm = y / screen.get_height()

                # Buscar la bounding box más cercana a las coordenadas del ratón
                min_distance = float('inf')
                min_index = None
                for i, detection in enumerate(detections):
                    bbox = detection['bbox']
                    bbox_x = bbox[0] + bbox[2] / 2
                    bbox_y = bbox[1] + bbox[3] / 2
                    distance = (x_norm - bbox_x) ** 2 + (y_norm - bbox_y) ** 2
                    if distance < min_distance:
                        min_distance = distance
                        min_index = i

                # Eliminar la bounding box seleccionada del JSON
                deleted_bbox.append(detections.pop(min_index))
                delete_bbox = False

    # Restaurar el último elemento eliminado
    if restored_bbox:
        detections.append(restored_bbox)
        restored_bbox = None


# Cerrar Pygame
pygame.quit()